<a href="https://colab.research.google.com/github/SWazniewicz/UMwF-1-/blob/main/Zadanie_3_UMwF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Stwórz model generujący sygnały kupna i sprzedaży na rynku
Decyzje ma generować jeden z algorytmów uczenia maszynowego, po optymalizacji hiperparametrów
Zrób backtest wybranego modelu, użyj biblioteki przystosowanej do backtestów
Pamiętaj o wnioskach i wizualizacji wyników
4 pkt z oceny będą zależne od wyników inwestycji
Inwestujesz w wylosowaną spółkę przez określony okres, ale można wykorzystać także inne dane niż historyczne (np. obliczone wskaźniki)

Strategia inwestycyjna (decyzje wejścia i wyjścia) dla spółki  Microsoft (MSFT), test w okresie od 01.01.2024 - 06.05.2024


In [2]:
!pip install yfinance backtesting scikit-learn ta --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 kB 5.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
import ta

/usr/local/lib/python3.12/dist-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [4]:
symbol = "MSFT"
start_train = "2020-01-01"
end_all = "2024-05-06"
test_start = "2024-01-01"
test_end = "2024-05-06"

data = yf.download(symbol, start=start_train, end=end_all)
data.head()

/tmp/ipython-input-1817777978.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, start=start_train, end=end_all)
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,MSFT,MSFT,MSFT,MSFT,MSFT
Date,,,,,
2020-01-02,152.505692,152.610135,150.331386,150.758649,22622100
2020-01-03,150.606750,151.869563,150.075043,150.321918,21116200
2020-01-06,150.995972,151.062442,148.603275,149.144487,20813700
2020-01-07,149.619263,151.603675,149.372403,151.271365,21634100
2020-01-08,152.002441,152.676579,149.970552,150.901040,27746500


In [5]:
print(data.columns)

MultiIndex([( 'Close', 'MSFT'),
            (  'High', 'MSFT'),
            (   'Low', 'MSFT'),
            (  'Open', 'MSFT'),
            ('Volume', 'MSFT')],
           names=['Price', 'Ticker'])


In [6]:
data.columns = data.columns.get_level_values(0)

In [7]:
print(data.columns)
print(data.head())

Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')
Price            Close        High         Low        Open    Volume
Date                                                                
2020-01-02  152.505692  152.610135  150.331386  150.758649  22622100
2020-01-03  150.606750  151.869563  150.075043  150.321918  21116200
2020-01-06  150.995972  151.062442  148.603275  149.144487  20813700
2020-01-07  149.619263  151.603675  149.372403  151.271365  21634100
2020-01-08  152.002441  152.676579  149.970552  150.901040  27746500


In [8]:
df = data.copy()

# Stopy zwrotu
df["return_1d"] = df["Close"].pct_change()
df["return_5d"] = df["Close"].pct_change(5)
df["return_10d"] = df["Close"].pct_change(10)

# Średnie kroczące
df["sma_10"] = df["Close"].rolling(10).mean()
df["sma_20"] = df["Close"].rolling(20).mean()
df["sma_50"] = df["Close"].rolling(50).mean()

# RSI
df["rsi_14"] = ta.momentum.rsi(df["Close"], window=14)

# MACD
macd = ta.trend.MACD(df["Close"])
df["macd"] = macd.macd()
df["macd_signal"] = macd.macd_signal()
df["macd_diff"] = macd.macd_diff()

# Wolumen
df["vol_sma_10"] = df["Volume"].rolling(10).mean()
df["vol_sma_20"] = df["Volume"].rolling(20).mean()

df.dropna(inplace=True)
df.head()

Price,Close,High,Low,Open,Volume,return_1d,return_5d,return_10d,sma_10,sma_20,sma_50,rsi_14,macd,macd_signal,macd_diff,vol_sma_10,vol_sma_20
Date,,,,,,,,,,,,,,,,,
2020-03-13,151.218018,154.150409,133.985461,140.431010,92727400,0.142169,-0.016959,-0.019628,152.212941,159.468943,160.396544,46.398604,-5.294388,-3.175328,-2.119060,69125970.0,62020600.0
2020-03-16,128.929962,142.192371,128.530093,133.290466,87905900,-0.147390,-0.100916,-0.216274,148.655037,157.116123,159.925029,36.896049,-6.741319,-3.888526,-2.852793,70813480.0,65258420.0
2020-03-17,139.545609,140.431031,128.530096,133.290470,81059800,0.082337,-0.089174,-0.109051,146.947018,155.204837,159.703806,42.894915,-6.951298,-4.501081,-2.450217,71751760.0,67921800.0
2020-03-18,133.671280,139.002904,128.549128,131.386306,81593200,-0.042096,-0.086116,-0.176781,144.076509,152.973174,159.357312,40.595151,-7.505201,-5.101905,-2.403297,74929640.0,70501585.0
2020-03-19,135.870636,142.954062,132.338431,135.927758,85922700,0.016453,0.026248,-0.141697,141.833429,150.987623,159.082340,41.852145,-7.678195,-5.617163,-2.061032,78740180.0,72954600.0


In [9]:
df["future_return_1d"] = df["Close"].pct_change().shift(-1)
df["target"] = (df["future_return_1d"] > 0).astype(int)
df.dropna(inplace=True)

In [10]:
test_start = "2024-01-01"
test_end = "2024-05-06"

feature_cols = [
    "return_1d", "return_5d", "return_10d",
    "sma_10", "sma_20", "sma_50",
    "rsi_14",
    "macd", "macd_signal", "macd_diff",
    "vol_sma_10", "vol_sma_20"
]

X = df[feature_cols]
y = df["target"]

train_mask = df.index < test_start
test_mask = (df.index >= test_start) & (df.index <= test_end)

X_train, y_train = X[train_mask], y[train_mask]
X_test, y_test = X[test_mask], y[test_mask]

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from scipy.stats import randint

tscv = TimeSeriesSplit(n_splits=5)

rf = RandomForestClassifier(random_state=42, n_jobs=-1)

param_distributions = {
    "n_estimators": randint(50, 300),
    "max_depth": randint(3, 20),
    "min_samples_split": randint(2, 20),
    "min_samples_leaf": randint(1, 10),
    "max_features": ["sqrt", "log2", None]
}

search = RandomizedSearchCV(
    rf,
    param_distributions=param_distributions,
    n_iter=30,
    scoring="accuracy",
    cv=tscv,
    n_jobs=-1,
    random_state=42,
    verbose=1
)

search.fit(X_train, y_train)
best_model = search.best_estimator_
print("Najlepsze parametry:", search.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Najlepsze parametry: {'max_depth': 15, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 8, 'n_estimators': 58}


In [12]:
df["signal_proba"] = best_model.predict_proba(X)[:, 1]
df["signal"] = (df["signal_proba"] > 0.55).astype(int)

In [13]:
from backtesting import Backtest, Strategy


bt_data = df.copy()
bt_data = bt_data[["Open", "High", "Low", "Close", "Volume", "signal"]]
bt_data_test = bt_data.loc[test_start:test_end].copy()

In [14]:
from backtesting import Strategy

class MLSignalStrategy(Strategy):
    def init(self):
        pass

    def next(self):
        signal = self.data.signal[-1]

        # Jeśli sygnał = 1 → chcemy być w pozycji long
        if signal == 1:
            if not self.position:
                self.buy()

        # Jeśli sygnał = 0 → wychodzimy z pozycji
        else:
            if self.position:
                self.position.close()

In [15]:
bt = Backtest(
    bt_data_test,
    MLSignalStrategy,
    cash=10000,
    commission=0.001,
    exclusive_orders=True
)

stats = bt.run()
stats

Backtest.run:   0%|          | 0/84 [00:00<?, ?bar/s]

,0
Start,2024-01-02 00:00:00
End,2024-05-02 00:00:00
Duration,121 days 00:00:00
Exposure Time [%],8.235294
Equity Final [$],10324.843248
Equity Peak [$],10352.01454
Commissions [$],60.467231
Return [%],3.248432
Buy & Hold Return [%],7.47046
Return (Ann.) [%],9.941181


Backtest strategii został przeprowadzony przy użyciu biblioteki backtesting.py, z kapitałem początkowym 10 000 USD oraz prowizją transakcyjną 0.1% od każdej transakcji. Zastosowano założenie jednej otwartej pozycji w danym czasie. Strategia osiągnęła dodatnią stopę zwrotu na poziomie 3.25% w analizowanym okresie, przy bardzo niskim maksymalnym obsunięciu kapitału wynoszącym 1.32%. Wysoki współczynnik Sharpe’a (2.06) wskazuje na korzystny stosunek zysku do ryzyka. W porównaniu do strategii „kup i trzymaj”, model osiągnął niższą stopę zwrotu, jednak charakteryzował się znacznie mniejszą ekspozycją na rynek oraz niższą zmiennością portfela.

In [16]:
bt.plot()

GridPlot(id='p1329', ...)

Wizualizacja backtestu potwierdza konserwatywny charakter strategii opartej na modelu uczenia maszynowego. Strategia generowała niewielką liczbę transakcji, unikała długotrwałych spadków cen oraz charakteryzowała się niskim obsunięciem kapitału.